# Artificial Neural Network and Deep Learning 
## Challenge 1: Image Classification
### Team:   Bergamasco Alex 10521973

This is the second approach that i used in this challenge.
I started adding as the first part of the network a VGG pretrained model, reaching an accuracy of 0.87. After some parameters tuning, I reached 0.92 accuracy. Then I tried different pretrained networks (InceptionResnetV2, InceptionV3, Xception and NASNet Large) reaching with the latest one an accuracy of 0.97 after tuning parameters.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import numpy as np
import tensorflow as tf

SEED = 1234

tf.random.set_seed(SEED)
import os

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    print(e)
    
cwd = os.getcwd()

### Validation set already created in My_CNN file

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=40,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        shear_range=0.2,
                                        zoom_range=[0.8, 1.25],
                                        horizontal_flip=True,
                                        vertical_flip=False,
                                        fill_mode='reflect',
                                        brightness_range=[0.5, 1.5],
                                        cval=0,
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)

valid_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
dataset_dir = os.path.join('/kaggle/input/splitted-data/Classification_Dataset')

bs = 4

img_h = 299 
img_w = 299 

num_classes=20

input_shape = (img_h, img_w, 3)

decide_class_indices = True
if decide_class_indices:
    classes = [ 'owl',
                'galaxy',
                'lightning',
                'wine-bottle',
                't-shirt',
                'waterfall',
                'sword',
                'school-bus',
                'calculator',
                'sheet-music',
                'airplanes',
                'lightbulb',
                'skyscraper',
                'mountain-bike',
                'fireworks',
                'computer-monitor',
                'bear',
                'grand-piano',
                'kangaroo',
                'laptop']   
else:
    classes=None

training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=bs,
                                               classes=classes,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED) 

validation_dir = os.path.join(dataset_dir, 'validation')
valid_gen = valid_data_gen.flow_from_directory(validation_dir,
                                               batch_size=bs, 
                                               classes=classes,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)

In [ ]:
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

train_dataset = train_dataset.repeat()


valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

valid_dataset = valid_dataset.repeat()

### Creating the model

Steps:
- Import VGG16 and added Fully Connected at the end to get our classification
- Trials with adding others layers (dropout, dense, batchnorm)
- Import InceptionResNetV2
- Same trials
- Import InceptionV3
- Same trials
- Import NASNetLarge
- Same trials

At the end, NASNet performs better than others with this configuration.

I commented the code in which I perform finetuning in the VGG16 since I perform it only in the VGG.

In [ ]:
# VGG16
#pretrained_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

# INCEPTION
#pretrained_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3), classes=20)

# INCEPTIONRESNETV2
#pretrained_model = tf.keras.applications.InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(img_h, img_w, 3), pooling='avg', classes=20)

# XCEPTION
#pretrained_model = tf.keras.applications.xception.Xception(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3), classes=20)

# NASNET
pretrained_model = tf.keras.applications.nasnet.NASNetLarge(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3), classes=20)

In [ ]:
# Setting all the pretrained_model weights fixed
for layer in pretrained_model.layers:
    layer.trainable = False
    
    
'''
finetuning = True

if finetuning:
    freeze_until = 15 
    
    for layer in pretrained_model.layers[:freeze_until]:
        layer.trainable = False
else:
    pretrained_model.trainable = False
'''


model = tf.keras.Sequential()
model.add(pretrained_model)
model.add(tf.keras.layers.GlobalAveragePooling2D())
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=1024, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [ ]:
model.summary()

### Optimization Parameters

After some attempts with the VGG16, considering that I set "trainable" also some of the last layers of the pretrained network, I use learning rate smaller than the original one with the aim of not changing a lot the values.
It improves the accuracy.

In [ ]:
loss = tf.keras.losses.CategoricalCrossentropy()

lr = 1e-3     # For VGG16 with some trainable layers, lr = 1e-4

optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

metrics = ['accuracy']

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

### Start Training the model

In [ ]:
model.fit(x=train_dataset,
          epochs=10, 
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen))

### Get predictions on the test set

In [ ]:
from datetime import datetime

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
from PIL import Image
test_dir = os.path.join(cwd, 'Classification_Dataset/test')
image_filenames = next(os.walk(test_dir))[2]

results = {}
for image_name in image_filenames:
    img = Image.open(os.path.join(test_dir,'{}').format(image_name)).convert('RGB')
    img = img.resize((img_h, img_w))
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, 0) 
    
    out_softmax = model.predict(x=img_array / 255.)
    
    predicted_class = np.argmax(out_softmax, -1)
    
    predicted_class = predicted_class[0]
    
    results[image_name] = predicted_class
 
create_csv(results)